1. **Preparing Process**

---



In [1]:
import random
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import sys
sys.path.append('/home/users/xt37/LQ-Nets')
import resnet
from resnet import ResNet
from time import time
import os

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

In [2]:
print('DEVICE:', DEVICE)

DEVICE: cuda


In [3]:
def print_and_log(message, file):
    print(message)
    file.write(message + '\n')

class ApplyTransform(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        return self.transform(x), y

    def __len__(self):
        return len(self.dataset)

# normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

TRANSFORM_TRAIN = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    normalize,
])
TRANSFORM_TEST = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

TRAIN_SET = datasets.CIFAR10(root='./data', train=True, download=True, transform=TRANSFORM_TRAIN)
TEST_SET = datasets.CIFAR10(root='./data', train=False, download=True, transform=TRANSFORM_TEST)






100%|███████████████████████████████████████| 170498071/170498071 [00:02<00:00, 68047730.15it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
def test(model: nn.Module, loader: DataLoader):
    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    test_correct = 0
    test_total = 0
    start_time = time()
    start_memory = torch.cuda.memory_allocated(DEVICE) / 1024 if DEVICE.type == 'cuda' else 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            test_correct += (outputs.argmax(1) == targets).sum().item()
            test_total += targets.size(0)

    latency = time() - start_time
    memory_usage = torch.cuda.memory_allocated(DEVICE) / 1024 if DEVICE.type == 'cuda' else 0  # kB
    test_memory_usage = memory_usage - start_memory

    test_loss /= len(loader)
    test_acc = test_correct / test_total
    return test_loss, test_acc, latency, test_memory_usage

In [5]:
def train(w_nbits, a_nbits, lambda_val, lr=0.1, weight_decay=1e-3,
          optimizer_type='sgd', epochs=200, batch_size=128):

    # import resnet20
    # model = resnet20.ResNetCIFAR(w_nbits=w_nbits, a_nbits=a_nbits).to(DEVICE)

    model = ResNet(w_nbits=w_nbits, a_nbits=a_nbits, lambda_val=lambda_val).to(DEVICE)

    ckpt_base_filename = (
        '/home/users/xt37/LQ-Nets/resnet20_cifar'
        + ('' if w_nbits is None else f'_wq{w_nbits}')
        + ('' if a_nbits is None else f'_aq{a_nbits}')
        + ('' if lambda_val is None else f'_lambda{lambda_val}')
        + '.pt'
    )
    log_filename = ckpt_base_filename.replace('.pt', '.txt')
    last_saved_filename = None

    train_loader = DataLoader(TRAIN_SET, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TEST_SET, batch_size=batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()

    if optimizer_type.lower() == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_type.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type.lower() == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type.lower() == 'adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100, 150], gamma=0.1)

    best_test_acc = 0.0
    save_threshold_epoch = min(50, epochs // 3)


    with open(log_filename, 'w') as log_file:
        print_and_log(f'\nQuantization: weight={w_nbits} activation={a_nbits} threshold={lambda_val}, Using device: {DEVICE}', log_file)

        start_memory = torch.cuda.memory_allocated(DEVICE) / 1024 if DEVICE.type == 'cuda' else 0

        for epoch in range(epochs):
            print_and_log(f'\nEpoch {epoch+1}', log_file)
            model.train()

            train_loss = 0
            train_correct = 0
            train_total = 0

            start_time = time()

            for inputs, targets in tqdm(train_loader, desc='Train', unit='batch', ascii=True, dynamic_ncols=True):
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                train_correct += (outputs.argmax(1) == targets).sum().item()
                train_total += targets.size(0)

            train_loss /= len(train_loader)
            train_acc = train_correct / train_total

            train_latency = time() - start_time
            end_memory = torch.cuda.memory_allocated(DEVICE) / 1024 if DEVICE.type == 'cuda' else 0
            train_memory_usage = end_memory - start_memory

            print_and_log(f'LR: {optimizer.param_groups[0]["lr"]:.4e}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train Latency: {train_latency:.2f}s, Memory Usage: {train_memory_usage}KB', log_file)

            model.eval()
            test_loss, test_acc, test_latency, test_memory_usage = test(model, test_loader)
            print_and_log(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Latency: {test_latency:.2f}s, Test Memory Usage: {test_memory_usage}KB', log_file)

            scheduler.step()

            if epoch >= save_threshold_epoch:
                if test_acc > best_test_acc:
                    best_test_acc = test_acc
                    print('Saving best model ...')
                    if last_saved_filename is not None and last_saved_filename != ckpt_base_filename:
                        os.remove(last_saved_filename)
                    ckpt_filename = ckpt_base_filename.replace('.pt', '') + f'epoch_{epoch+1}_' + '.pt'
                    torch.save(model.state_dict(), ckpt_filename)
                    last_saved_filename = ckpt_filename

    print('Saving final epoch model ...')
    torch.save(model.state_dict(), ckpt_base_filename)

In [6]:
train(w_nbits=3, a_nbits=None, lambda_val=0.004)


Quantization: weight=3 activation=None threshold=0.004, Using device: cuda

Epoch 1


Train: 100%|###############################################| 391/391 [00:30<00:00, 12.85batch/s]


LR: 1.0000e-01, Train Loss: 1.6442, Train Acc: 0.3862, Train Latency: 30.43s, Memory Usage: 19750.0KB
Test Loss: 1.2924, Test Acc: 0.5252, Test Latency: 3.23s, Test Memory Usage: 194.0KB

Epoch 2


Train: 100%|#############################| 391/391 [00:35<00:00, 11.07batch/s]                  


LR: 1.0000e-01, Train Loss: 1.1569, Train Acc: 0.5827, Train Latency: 35.32s, Memory Usage: 19750.0KB
Test Loss: 1.1923, Test Acc: 0.5706, Test Latency: 3.68s, Test Memory Usage: 194.0KB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.1781, Train Acc: 0.9409, Train Latency: 28.15s, Memory Usage: 19750.0KB
Test Loss: 0.3372, Test Acc: 0.8908, Test Latency: 3.85s, Test Memory Usage: 194.0KB

Epoch 199


Train: 100%|##########################| 391/391 [00:34<00:00, 11.26batch/s]


LR: 1.0000e-04, Train Loss: 0.1753, Train Acc: 0.9417, Train Latency: 34.73s, Memory Usage: 19750.0KB
Test Loss: 0.3338, Test Acc: 0.8895, Test Latency: 4.22s, Test Memory Usage: 194.0KB

Epoch 200


Train: 100%|##########################| 391/391 [00:32<00:00, 11.88batch/s]


LR: 1.0000e-04, Train Loss: 0.1797, Train Acc: 0.9406, Train Latency: 32.91s, Memory Usage: 19750.0KB
Test Loss: 0.3383, Test Acc: 0.8843, Test Latency: 4.23s, Test Memory Usage: 194.0KB
Saving final epoch model ...


In [7]:
train(w_nbits=2, a_nbits=None, lambda_val=0.001)


Quantization: weight=2 activation=None threshold=0.001, Using device: cuda

Epoch 1


Train: 100%|##########################| 391/391 [00:29<00:00, 13.09batch/s]


LR: 1.0000e-01, Train Loss: 1.6257, Train Acc: 0.3985, Train Latency: 29.87s, Memory Usage: 3110.0KB
Test Loss: 1.5442, Test Acc: 0.4613, Test Latency: 2.48s, Test Memory Usage: 194.0KB

Epoch 2


Train: 100%|##########################| 391/391 [00:29<00:00, 13.15batch/s]


LR: 1.0000e-01, Train Loss: 1.1489, Train Acc: 0.5868, Train Latency: 29.73s, Memory Usage: 3110.0KB
Test Loss: 1.3718, Test Acc: 0.5438, Test Latency: 3.06s, Test Memory Usage: 194.0KB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.0917, Train Acc: 0.9717, Train Latency: 23.33s, Memory Usage: 3110.0KB
Test Loss: 0.2931, Test Acc: 0.9077, Test Latency: 2.44s, Test Memory Usage: 194.0KB

Epoch 199


Train: 100%|##########################| 391/391 [00:23<00:00, 16.72batch/s]


LR: 1.0000e-04, Train Loss: 0.0889, Train Acc: 0.9723, Train Latency: 23.40s, Memory Usage: 3110.0KB
Test Loss: 0.2980, Test Acc: 0.9061, Test Latency: 2.80s, Test Memory Usage: 194.0KB

Epoch 200


Train: 100%|##########################| 391/391 [00:24<00:00, 15.94batch/s]


LR: 1.0000e-04, Train Loss: 0.0911, Train Acc: 0.9719, Train Latency: 24.53s, Memory Usage: 3110.0KB
Test Loss: 0.2902, Test Acc: 0.9105, Test Latency: 2.78s, Test Memory Usage: 194.0KB
Saving final epoch model ...


In [8]:
train(w_nbits=2, a_nbits=None, lambda_val=0.004)


Quantization: weight=2 activation=None threshold=0.004, Using device: cuda

Epoch 1


Train: 100%|##########################| 391/391 [00:26<00:00, 14.88batch/s]


LR: 1.0000e-01, Train Loss: 1.6270, Train Acc: 0.3970, Train Latency: 26.28s, Memory Usage: 3110.0KB
Test Loss: 1.6971, Test Acc: 0.4563, Test Latency: 2.85s, Test Memory Usage: 194.0KB

Epoch 2


Train: 100%|##########################| 391/391 [00:26<00:00, 14.81batch/s]


LR: 1.0000e-01, Train Loss: 1.1651, Train Acc: 0.5823, Train Latency: 26.40s, Memory Usage: 3110.0KB
Test Loss: 1.6915, Test Acc: 0.4851, Test Latency: 2.81s, Test Memory Usage: 194.0KB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.1709, Train Acc: 0.9440, Train Latency: 26.41s, Memory Usage: 3092.0KB
Test Loss: 0.3201, Test Acc: 0.8953, Test Latency: 2.78s, Test Memory Usage: 194.0KB

Epoch 199


Train: 100%|##########################| 391/391 [00:26<00:00, 14.89batch/s]


LR: 1.0000e-04, Train Loss: 0.1702, Train Acc: 0.9431, Train Latency: 26.26s, Memory Usage: 3092.0KB
Test Loss: 0.3162, Test Acc: 0.8953, Test Latency: 2.79s, Test Memory Usage: 194.0KB

Epoch 200


Train: 100%|##########################| 391/391 [00:26<00:00, 14.87batch/s]


LR: 1.0000e-04, Train Loss: 0.1690, Train Acc: 0.9442, Train Latency: 26.30s, Memory Usage: 3092.0KB
Test Loss: 0.3368, Test Acc: 0.8910, Test Latency: 2.80s, Test Memory Usage: 194.0KB
Saving final epoch model ...


In [9]:
train(w_nbits=1, a_nbits=None, lambda_val=0.001)


Quantization: weight=1 activation=None threshold=0.001, Using device: cuda

Epoch 1


Train: 100%|##########################| 391/391 [00:24<00:00, 16.11batch/s]


LR: 1.0000e-01, Train Loss: 1.5975, Train Acc: 0.4180, Train Latency: 24.27s, Memory Usage: 3110.0KB
Test Loss: 1.5109, Test Acc: 0.4576, Test Latency: 2.37s, Test Memory Usage: 194.0KB

Epoch 2


Train: 100%|##########################| 391/391 [00:23<00:00, 16.99batch/s]


LR: 1.0000e-01, Train Loss: 1.1510, Train Acc: 0.5881, Train Latency: 23.02s, Memory Usage: 3110.0KB
Test Loss: 1.7684, Test Acc: 0.4534, Test Latency: 2.10s, Test Memory Usage: 194.0KB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.2033, Train Acc: 0.9309, Train Latency: 24.08s, Memory Usage: 3110.0KB
Test Loss: 0.3515, Test Acc: 0.8832, Test Latency: 2.39s, Test Memory Usage: 194.0KB

Epoch 199


Train: 100%|##########################| 391/391 [00:22<00:00, 17.71batch/s]


LR: 1.0000e-04, Train Loss: 0.2034, Train Acc: 0.9320, Train Latency: 22.09s, Memory Usage: 3110.0KB
Test Loss: 0.3578, Test Acc: 0.8819, Test Latency: 2.09s, Test Memory Usage: 194.0KB

Epoch 200


Train: 100%|##########################| 391/391 [00:23<00:00, 16.99batch/s]


LR: 1.0000e-04, Train Loss: 0.2058, Train Acc: 0.9316, Train Latency: 23.01s, Memory Usage: 3110.0KB
Test Loss: 0.3811, Test Acc: 0.8734, Test Latency: 2.37s, Test Memory Usage: 194.0KB
Saving final epoch model ...


In [10]:
train(w_nbits=1, a_nbits=None, lambda_val=0.004)


Quantization: weight=1 activation=None threshold=0.004, Using device: cuda

Epoch 1


Train: 100%|##########################| 391/391 [00:24<00:00, 16.09batch/s]


LR: 1.0000e-01, Train Loss: 1.6290, Train Acc: 0.3959, Train Latency: 24.30s, Memory Usage: 3110.0KB
Test Loss: 1.7522, Test Acc: 0.3924, Test Latency: 2.37s, Test Memory Usage: 194.0KB

Epoch 2


Train: 100%|##########################| 391/391 [00:22<00:00, 17.04batch/s]


LR: 1.0000e-01, Train Loss: 1.1903, Train Acc: 0.5690, Train Latency: 22.95s, Memory Usage: 3110.0KB
Test Loss: 1.3331, Test Acc: 0.5323, Test Latency: 2.10s, Test Memory Usage: 194.0KB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.2065, Train Acc: 0.9315, Train Latency: 22.57s, Memory Usage: 3110.0KB
Test Loss: 0.3397, Test Acc: 0.8879, Test Latency: 2.37s, Test Memory Usage: 194.0KB

Epoch 199


Train: 100%|##########################| 391/391 [00:21<00:00, 18.46batch/s]


LR: 1.0000e-04, Train Loss: 0.2071, Train Acc: 0.9299, Train Latency: 21.18s, Memory Usage: 3110.0KB
Test Loss: 0.3604, Test Acc: 0.8816, Test Latency: 2.40s, Test Memory Usage: 194.0KB

Epoch 200


Train: 100%|##########################| 391/391 [00:21<00:00, 17.93batch/s]


LR: 1.0000e-04, Train Loss: 0.2081, Train Acc: 0.9300, Train Latency: 21.81s, Memory Usage: 3110.0KB
Test Loss: 0.3643, Test Acc: 0.8790, Test Latency: 2.35s, Test Memory Usage: 194.0KB
Saving final epoch model ...


In [6]:
for w_nbits in [3, 2, 1]:
    for lambda_val in [0.001, 0.004]:
        train(w_nbits=w_nbits, a_nbits=None, lambda_val=lambda_val)


Quantization: weight=3 activation=None threshold=0.001, Using device: cuda

Epoch 1


Train: 100%|###############################| 391/391 [00:25<00:00, 15.09batch/s]


LR: 1.0000e-01, Train Loss: 1.5751, Train Acc: 0.4182, Train Latency: 25.91s, Memory Usage: 19750.0KB
Test Loss: 1.5508, Test Acc: 0.4768, Test Latency: 3.32s, Test Memory Usage: 194.0KB

Epoch 2


Train: 100%|###############################| 391/391 [00:30<00:00, 12.75batch/s]


LR: 1.0000e-01, Train Loss: 1.1141, Train Acc: 0.5974, Train Latency: 30.67s, Memory Usage: 19750.0KB
Test Loss: 1.2724, Test Acc: 0.5723, Test Latency: 3.56s, Test Memory Usage: 194.0KB

Epoch 3


...

LR: 1.0000e-01, Train Loss: 0.6590, Train Acc: 0.7737, Train Latency: 30.20s, Memory Usage: 3110.0KB
Test Loss: 0.8781, Test Acc: 0.7031, Test Latency: 3.54s, Test Memory Usage: 194.0KB

Epoch 42


Train: 100%|###############################| 391/391 [00:27<00:00, 14.06batch/s]


LR: 1.0000e-01, Train Loss: 0.6628, Train Acc: 0.7717, Train Latency: 27.81s, Memory Usage: 3110.0KB
Test Loss: 1.1768, Test Acc: 0.6332, Test Latency: 3.20s, Test Memory Usage: 194.0KB

Epoch 43


Train:  68%|####################9          | 264/391 [00:19<00:10, 12.58batch/s]

: 